In [3]:
%pip install --upgrade --quiet devtools

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import json
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

load_dotenv()

/Users/afawaz/Documents/misc/ml/bot/.conda/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [3]:
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device="mps")

In [5]:
filename = "./Chatbot_data_filtered-V2.json"
pc = Pinecone()
index = pc.Index("catalog-v2-768")

i = j = 0
batch_size = 100
upserted_count = 0
texts, ids, embeds, metadata = [], [], [], []
with open(filename) as file:
    for line in file:
        if j % 1000 == 0:
            print(f"Iteration: {j}. Upserted: ~{upserted_count}")
        if i % batch_size == 0 and texts:
            embeds = model.encode(texts)
            upserted_count += len(embeds)
            index.upsert(vectors=zip(ids, embeds, metadata))
            texts, ids, embeds, metadata = [], [], [], []
        doc = json.loads(line.rstrip())
        if (
            doc.get("title_en")
            and doc.get("description")
            and doc.get("seller_price")
            and doc.get("image_url")
            and doc.get("product_url")
            and doc.get("is_returnable")
        ):
            specs = doc.get("specifications") or []
            if not specs:
                continue
            specs_text = ""
            specs = json.loads(specs)
            for spec in specs:
                for k, v in spec.items():
                    k = spec.get("name")
                    v = spec.get("value")
                    specs_text += f"{k} : {v}" + "\n"
            ids.append(doc["offer_code"])
            m = {
                "sku": doc.get("sku_config"),
                "title": doc.get("title_en"),
                "image_url": doc.get("image_url"),
                "product_url": f"https://www.{doc['product_url']}",
                "specs": specs_text
            }
            title = doc.get("title_en", "Unavailable")
            offer_price = doc.get("seller_price", "Unavailable")
            description = doc.get("description", "Unavailable")
            highlights = doc.get("highlights", "Unavailable")
            is_returnable = doc.get("is_returnable", "Unavailable")
            rating = doc.get("product_rating", "Unavailable")
            warranty = doc.get("warranty", "Unavailable")
            image_url = doc.get("image_url") + "?width=240"
            text = (
                f"Title: {title}\n"
                f"Price: {offer_price}\n"
                f"Description: {description}.\n"
                f"Highlights: {highlights}.\n"
                f"is_returnable: {is_returnable}\n"
                f"rating: {rating}"
                f"warranty: {warranty}"
                f"is_on_deal: {doc.get('deal_flag')} or 'False'"
            )
            m["text"] = text
            metadata.append({**m})
            texts.append(text)
            i += 1
        j += 1

Iteration: 0. Upserted: ~0


KeyboardInterrupt: 